In [8]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql import functions as f

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("GCSExample_ass2")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

#  Google Storage File Path
gcs_file_paths = ['gs://data_de2022_2093373/olist_customers_dataset.csv', 
                'gs://data_de2022_2093373/olist_products_dataset.csv', 
                'gs://data_de2022_2093373/olist_sellers_dataset.csv']

# Create data frames
customers = spark.read.format("csv").option("header", "true") \
        .load(gcs_file_paths[0])
customers.printSchema()
    
products = spark.read.format("csv").option("header", "true") \
        .load(gcs_file_paths[1])
products.printSchema()
    
sellers = spark.read.format("csv").option("header", "true") \
        .load(gcs_file_paths[2])
sellers.printSchema()
    


root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: string (nullable = true)
 |-- product_description_lenght: string (nullable = true)
 |-- product_photos_qty: string (nullable = true)
 |-- product_weight_g: string (nullable = true)
 |-- product_length_cm: string (nullable = true)
 |-- product_height_cm: string (nullable = true)
 |-- product_width_cm: string (nullable = true)

root
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: string (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)



### Do preprocessing here

#### Create new column with regions of Brazil and which states belong to them

In [9]:
southeast = ['SP', 'RJ', 'ES','MG']
northeast = ['MA', 'PI', 'CE', 'RN', 'PE', 'PB', 'SE', 'AL', 'BA']
north = ['AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC']
centerwest = ['MT', 'GO', 'MS' ,'DF' ]
south = ['SC', 'RS', 'PR']

In [10]:
customers = customers.withColumn("customer_zip_code_prefix",customers.customer_zip_code_prefix.cast('int')) # change to integer
customers = customers.withColumn('customer_region', f.when(f.col('customer_state').isin(southeast), f.lit('Southeast'))
                                           .when(f.col('customer_state').isin(northeast), f.lit('Northeast'))
                                           .when(f.col('customer_state').isin(north), f.lit('North'))
                                           .when(f.col('customer_state').isin(centerwest), f.lit('Centerwest'))
                                           .when(f.col('customer_state').isin(south), f.lit('South')))
customers = customers.withColumnRenamed("customer_id", "customer_order_id") # change name to better fit column purpose
customers = customers.sort("customer_state") # sort on the state's for better readability

In [11]:
sellers = sellers.withColumn("seller_zip_code_prefix",sellers.seller_zip_code_prefix.cast('int')) # change to integer
sellers = sellers.withColumn('seller_region', f.when(f.col('seller_state').isin(southeast), f.lit('Southeast'))
                                           .when(f.col('seller_state').isin(northeast), f.lit('Northeast'))
                                           .when(f.col('seller_state').isin(north), f.lit('North'))
                                           .when(f.col('seller_state').isin(centerwest), f.lit('Centerwest'))
                                           .when(f.col('seller_state').isin(south), f.lit('South')))
sellers = sellers.sort('seller_state') # sort on the state's for better readability

In [12]:
products = products.withColumnRenamed('product_name_lenght', 'product_name_length') # rename as there was a typo in the dataset
products = products.withColumnRenamed('product_description_lenght', 'product_description_length') # rename as there was a typo in the dataset
products = products.withColumn('product_name_length', products.product_name_length.cast('int')) # change to integer
products = products.withColumn('product_description_length', products.product_description_length.cast('int')) # change to integer
products = products.withColumn('product_photos_qty', products.product_photos_qty.cast('int')) # change to integer
products = products.withColumn('product_weight_g', products.product_weight_g.cast('int')) # change to integer
products = products.withColumn('product_length_cm', products.product_length_cm.cast('int')) # change to integer
products = products.withColumn('product_height_cm', products.product_height_cm.cast('int')) # change to integer
products = products.withColumn('product_width_cm', products.product_width_cm.cast('int')) # change to integer
products = products.sort('product_category_name') # sort on the product name's for better readability

### Saving to BigQuery through bucket

In [13]:
# Use the Cloud Storage bucket for temporary BigQuery export data used by the connector.
bucket = "de_jads_temp_2093373"  # use your bucket 
spark.conf.set('temporaryGcsBucket', bucket)

# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

# Saving the data to BigQuery
customers.write.format('bigquery') \
  .option('table', 'de2022-362707.assignment2.customers') \
  .mode("append") \
  .save()

sellers.write.format('bigquery') \
  .option('table', 'de2022-362707.assignment2.sellers') \
  .mode("append") \
  .save()

products.write.format('bigquery') \
  .option('table', 'de2022-362707.assignment2.products') \
  .mode("append") \
  .save()


In [ ]:
# Stop the spark context
spark.stop()

In [7]:
with open('D:\documents\YEAR6_JADS_M1\1_Data_Engineering\Assignment2\Data\olist_orders_dataset.csv') as f:
        lines = f.readlines()

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\documents\\YEAR6_JADS_M1\x01_Data_Engineering\\Assignment2\\Data\\olist_orders_dataset.csv'